In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from geopy.geocoders import Nominatim

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
%matplotlib inline

In [3]:
def read_tweets(filename):
    df = pd.read_csv(filename)
    df = df[['date','text','geo','id']]
    return df

In [4]:
df1 = read_tweets('data/kplc_tweets_KenyaPowerAlerts.csv')
df2 = read_tweets('data/kplc_tweets_KenyaPower_Care.csv')
df3 = read_tweets('data/kplc_tweets_KENYAPOWER.csv')
df4 = read_tweets('data/kplc_tweets_kplc.csv')
df5 = read_tweets('data/kplc_tweets_Kplc.csv')
df6 = read_tweets('data/kplc_tweets_KPLC.csv')

In [5]:
data = df1.append([df2,df3,df4,df5,df6])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27315 entries, 0 to 1820
Data columns (total 4 columns):
date    27315 non-null object
text    27315 non-null object
geo     32 non-null object
id      27292 non-null object
dtypes: object(4)
memory usage: 1.0+ MB


In [7]:
data2 = data.drop_duplicates()
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23652 entries, 0 to 1826
Data columns (total 4 columns):
date    23652 non-null object
text    23652 non-null object
geo     28 non-null object
id      23629 non-null object
dtypes: object(4)
memory usage: 923.9+ KB


In [8]:
text = data2['text'].values

In [9]:
def get_loc(text):
    locs = []
    words = [w.lower() for w in text.split(' ')]
    try:
        if 'road' in words:
            loc = words[words.index('road')-1] + ' road, kenya'
            locs.append(loc)
        if 'estate' in words:
            loc = words[words.index('estate')-1] + ' estate, kenya'
            locs.append(loc)
        if 'town' in words:
            loc = words[words.index('town')-1] + ' town, kenya'
            locs.append(loc)
        if 'area' in words:
            loc = words[words.index('area')-1] + ' area, kenya'
            locs.append(loc)
        if 'ward' in words:
            loc = words[words.index('ward')-1] + ' ward, kenya'
            locs.append(loc)
        if 'rd' in words:
            loc = words[words.index('rd')-1] + ' road, kenya'
            locs.append(loc)
        if 'in' in words:
            loc = words[words.index('in')+1] +', kenya'
            locs.append(loc)
        if 'on' in words:
            loc = words[words.index('on')+1] +', kenya'
            locs.append(loc)
        if 'location' in words:
            loc = words[words.index('location')-1] +', kenya'
            locs.append(loc)
        if 'county' in words:
            loc = words[words.index('county')-1] + ' county, kenya'
            locs.append(loc)
    except:
        loc = None
        locs.append(loc)
    return locs

data2['loc'] = data2['text'].apply(get_loc)
data3 = data2[data2.astype(str)['loc'] != '[]']
data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11227 entries, 0 to 1825
Data columns (total 5 columns):
date    11227 non-null object
text    11227 non-null object
geo     3 non-null object
id      11224 non-null object
loc     11227 non-null object
dtypes: object(5)
memory usage: 526.3+ KB


/home/rosina/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
locations = data3[['text','loc']].values

In [11]:
for i in np.random.randint(0,11196,20):
    print locations[i]
    

[ '@KenyaPower_Care No power in some houses at Omega along kangundo road. since yesterday 5p.m. Kindly assist'
 list(['some, kenya'])]
[ '@KenyaPower_Care @KenyaPower Still in darkness... What happened yesterday and today? South B - Balozi Area?'
 list(['darkness..., kenya'])]
[ '@KenyaPower help out in Kiambu, njathaine road messed up transformer. No lights'
 list(['njathaine road, kenya', 'kiambu,, kenya'])]
[ 'Stop pretending of already power in coast while rabai road in Nairobi is blackout from sonko rd. to jogoo road I wonder what your doing...'
 list(['rabai road, kenya', 'coast, kenya'])]
[ 'The last mile project has doubled connectivity in #Kenya in the last 2 years - PS Dr Eng. Joseph Njoroge at launch of #ejea3 @EnergyMinK ^EK'
 list(['#kenya, kenya'])]
[ '@KenyaPower @KenyaPower_Care Again no power in entire kikambala area. Ac 3917346'
 list(['entire, kenya'])]
[ '@KenyaPower only in kenya were we pay for blackout, restore power please at Mwiki, SDA chuch line, mtr no 141043

In [100]:
geolocator = Nominatim()
data4 = data3.iloc[:100]
data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 141
Data columns (total 5 columns):
date    100 non-null object
text    100 non-null object
geo     0 non-null object
id      100 non-null object
loc     100 non-null object
dtypes: object(5)
memory usage: 4.7+ KB


In [111]:
def get_coords(locs):
    correct_coords = None
    for loc in locs:
        location = geolocator.geocode(loc)
        if location == None:
            continue
        coords = [location.latitude, location.longitude]
        correct_coords = coords
#             # check 
#             if 'kenya' in str(geolocator.reverse("{}, {}".format(coords[0],coords[1])).address).split(', '):
#                 correct_coords = coords
    return correct_coords

locs = data4['loc'].values

In [115]:
for locations in locs:
    for l in locations:
        location = geolocator.geocode(l)
        if location == None:
            continue
        coords = [location.latitude, location.longitude]
        print '{} location, coordinates = {}'.format(l,coords)

ksouth, kenya location, coordinates = [-1.2621843, 36.8875606204204]
lavington., kenya location, coordinates = [-1.2805926, 36.7707077]
kariobangi, kenya location, coordinates = [-1.2499923, 36.8832942]
kariobangi, kenya location, coordinates = [-1.2499923, 36.8832942]
nakuru town, kenya location, coordinates = [-0.4999751, 36.3240798]
some, kenya location, coordinates = [-0.12417115, 34.3157355488373]
kiambu road, kenya location, coordinates = [-1.2307687, 36.8418821]
kasarini, kenya location, coordinates = [-1.196216, 36.8181255]
kawangware, kenya location, coordinates = [-1.2784631, 36.751643]
jamhuri,woodley., kenya location, coordinates = [-1.3056567, 36.7760046486656]
and, kenya location, coordinates = [1.4419683, 38.4313975]
our, kenya location, coordinates = [-1.3112059, 36.834068]
moja road, kenya location, coordinates = [-4.0633939, 39.6670989]
embakasi, kenya location, coordinates = [-1.3506088, 36.8941981]
huruma, kenya location, coordinates = [-1.25764595, 36.8725983529175

GeocoderTimedOut: Service timed out